In [1]:
import pandas as pd
import re
from lxml import etree
from pymystem3 import Mystem
import many_stop_words
ru_stopwords = many_stop_words.get_stop_words('ru').union(
    {'либо', 'б', 'срок', 'иной', 'преступление', 'наказание', 'мера', 
     'лишение', 'свобода', 'гражданин', 'размер', 'лицо'}
)

In [2]:
with open('data/RFCriminalCode.xml', 'r') as file:
    tree = etree.parse(file)

In [3]:
root = tree.getroot()

articles = pd.DataFrame(columns = ['number', 'section', 'section_name', 'chapter', 'chapter_name', 'name', 'body'])
sec = {'I':1, 'II':2, 'III':3, 'IV':4, 'V':5, 'VI':6, 'VII':7, 'VIII':8, 'IX':9, 'X':10, 'XI':11, 'XII':11}

for part in root.getchildren():
    for section in part.getchildren():
        for chapter in section.getchildren():
            for article in chapter.getchildren():
                articles = articles.append({
                    'number': float(article.get('number')),
                    'section': sec[section.get('number')],
                    'section_name': section.get('name'),
                    'chapter': float(chapter.get('number')),
                    'chapter_name': chapter.get('name'),
                    'name': article.get('name'),
                    'body': article.getchildren()[0].text
                }, ignore_index=True)

Вырезаем ссылки на другие статьи из текста статей

In [4]:
pattern = re.compile('\s\((\w|\s)*стать.*?\)')
articles['body'] = articles['body'].map(lambda x: re.sub(pattern, '', x))

Лемматизируем статьи

In [5]:
m = Mystem(entire_input=False)
articles['lemmas'] = articles['body'].map(m.lemmatize)

Выбрасываем стоп-слова

In [6]:
articles['lemmas'] = articles['lemmas'].map(lambda lemmas: list(filter(lambda lemma: lemma not in ru_stopwords, lemmas)))

In [7]:
articles.to_csv('tmp/criminal_code.csv', index=False)